1. imports

In [51]:
import torch
import torch.nn as nn
#modules , loss functions
import torch.nn.functional as F
#activations,functions having no parameters
import torch.optim as optim
#optimisation algos
from torch.utils.data import DataLoader
# data sets , gives mini batches
from torchvision import transforms , datasets
# many datasets

2.Create FullyConnected Network

In [52]:

class CNN(nn.Module):
  def __init__(self, input_channels=1, num_classes=10):
    super(CNN, self).__init__()
    self.conv1 = nn.  Conv2d(in_channels=1,out_channels=8, stride=(1,1), padding=(1,1), kernel_size=(3,3) )
    self.pool =nn.MaxPool2d(stride=(2,2), kernel_size=(2,2))#7*7 , assuming we are using pool twice
    self.conv2 = nn.Conv2d(in_channels=8,out_channels=16, stride=(1,1), padding=(1,1), kernel_size=(3,3) )
    self.fc1= nn.Linear(16*7*7,num_classes)

  def forward(self, x):
    x=F.relu(self.conv1(x))

    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x





Set device

In [53]:
device =("cuda" if torch.cuda.is_available() else "cpu")

Hyper Parameters

In [54]:
input_channels=1
num_classes=10
batch=64
learning_rate=0.01
epochs=1

Load Data

In [55]:
train_set = datasets.MNIST(download=True,train=True,root='dataset/', transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_set,batch_size=batch,shuffle=True)
test_set = datasets.MNIST(download=True,train=False,root='dataset/', transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_set,batch_size=batch,shuffle=True)

Intialize the network

In [56]:
model = CNN().to(device)

LOss and Optimizer

In [57]:
criterion = nn.CrossEntropyLoss()
optimizer =optim.Adam(model.parameters(), lr = learning_rate)

Train the network

In [58]:
for epoch in range(epochs):
  for batch_index, (data, targets) in enumerate(train_loader):
    data.to(device)
    targets.to(targets)
    scores = model(data)
    #calculate loss
    loss = criterion(scores, targets)
    #calculate bp
    optimizer.zero_grad()
    loss.backward()
    #update
    optimizer.step()

Check accuracy of our model

In [61]:
def check_accuracy(loader,model):
  if loader.dataset.train:
    print('*********training accuracy**********')
  else:
     print('*********testing accuracy**********')
  num_of_correct=0
  num_of_samples=0
  with torch.no_grad():
    for x, y in loader:
      x= x.to(device)
      y= y.to(device)
      scores = model(x)
      _ , predictions = scores.max(1)# 1 represent dimension (64, 10), takke maximum probability
      num_of_correct +=(predictions==y).sum() # y & predictions  is list , make sum of them
      num_of_samples+=predictions.size(0)

    print("accuracy is {}".format(num_of_correct/num_of_samples))

  model.train()

check_accuracy(train_loader,model )
check_accuracy(test_loader,model )


*********training accuracy**********
accuracy is 0.9803333282470703
*********testing accuracy**********
accuracy is 0.9807999730110168
